In [37]:
import os

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import make_scorer

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from scipy.stats import friedmanchisquare


from time import time

from tqdm import tqdm

import pickle
import json

# Clustering

In [2]:
K = list(range(2, 10))

In [3]:
df = pd.read_csv(r'./df_tfidf4395.csv')
df.head()

,act,afraid,age,ago,ah,ai,aint,air,aliv,alon,...,x2,ya,ye,yea,yeah,year,yesterday,yo,young,index
0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.028271,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0
1,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,1
2,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.085668,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,2
3,0.0,0.0,0.0,0.0,0.0,0.03901,0.0,0.254412,0.000000,0.000000,...,0.0,0.0,0.100178,0.0,0.035947,0.188197,0.0,0.0,0.027036,3
4,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.216461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,4


In [4]:
features = df.columns.tolist()[:-1]

In [5]:
data = df[features].values

## Kmeans

In [6]:
results = {}
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=905).fit(data)
    labels = kmeans.labels_
    results[k] = {
                  'db_index': davies_bouldin_score(data, labels),
                  'labels': kmeans.labels_,
                  'centres': kmeans.cluster_centers_
                 }
    print(k, davies_bouldin_score(data, labels))

2 8.5776640097168
3 7.602568532441171
4 6.5123734239046405
5 7.860200282523778
6 7.593222031160148
7 7.236792062867887
8 6.571751734390039
9 6.643355777503522


## LDA

In [7]:
n_top_words = 20

In [8]:
lda = LatentDirichletAllocation(n_components=4, 
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=905)

In [9]:
t0 = time()
lda.fit(data)
print("done in %0.3fs." % (time() - t0))

done in 2.418s.


In [10]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [11]:
print_top_words(lda, features, n_top_words)

Topic #0: la holi da shall prais glori mr bore lord king hi born power chain fame lil luck ooh sing high
Topic #1: dem doll di wild escap lone choru quick check like war say bout heart want deep day man come da
Topic #2: thou brown touch suck hot gold kill burn great fuck murder bag woah feel tree love bird deeper need wit
Topic #3: love know like got ll time ve oh come want na let say feel make way babi ca day life



# Classification

## Data preparing

In [12]:
with open(r'./lyrics_emotion[0-7480].json', 'r') as fn:
    emo_dict = json.load(fn)

In [13]:
data_cls = pd.read_csv('./df_4395.csv')
data_cls.head()

,index,song,year,artist,genre,lyrics,word_count,represent,represent_2nd,word_count_2nd
0,0,louder-flux-pavilion-doctor-p-remix,2012,dj-fresh,Electronic,I can't control this feeling Something's happe...,341.0,I ca n't control feel someth 's happen insid o...,I ca n't control feel someth 's happen insid s...,339
1,1,that-s-my-name,2009,akcent,Pop,In my heart I will keep you In my heart Foreve...,318.0,In heart I keep In heart forev In heart and on...,In heart I keep In heart forev In heart one li...,169
2,2,lemonade,2007,apologetix,Rock,"Oh, Mama, I've been cleared of my crimes and I...",250.0,Oh mama I 've clear crime I 'm law law put end...,Oh mama I 've clear crime I 'm law law put end...,119
3,3,return-of-the-hustle,2007,fabolous,Hip-Hop,Record mode! (Record mode!) Get your money in ...,681.0,record mode record mode get money air like yea...,record record get money air like yeah yeah tim...,338
4,4,so-sad,1974,george-harrison,Rock,Now the winter has come To eclipse out the sun...,163.0,now winter come To eclips sun that light love ...,winter come To sun light love sometim cold win...,72


In [14]:
data_cls['clustered_label'] = pd.Series(results[4]['labels'])
data_cls.head()

,index,song,year,artist,genre,lyrics,word_count,represent,represent_2nd,word_count_2nd,clustered_label
0,0,louder-flux-pavilion-doctor-p-remix,2012,dj-fresh,Electronic,I can't control this feeling Something's happe...,341.0,I ca n't control feel someth 's happen insid o...,I ca n't control feel someth 's happen insid s...,339,3
1,1,that-s-my-name,2009,akcent,Pop,In my heart I will keep you In my heart Foreve...,318.0,In heart I keep In heart forev In heart and on...,In heart I keep In heart forev In heart one li...,169,2
2,2,lemonade,2007,apologetix,Rock,"Oh, Mama, I've been cleared of my crimes and I...",250.0,Oh mama I 've clear crime I 'm law law put end...,Oh mama I 've clear crime I 'm law law put end...,119,2
3,3,return-of-the-hustle,2007,fabolous,Hip-Hop,Record mode! (Record mode!) Get your money in ...,681.0,record mode record mode get money air like yea...,record record get money air like yeah yeah tim...,338,0
4,4,so-sad,1974,george-harrison,Rock,Now the winter has come To eclipse out the sun...,163.0,now winter come To eclips sun that light love ...,winter come To sun light love sometim cold win...,72,1


In [15]:
emotions = ['Fear',
            'Sad',
            'Bored',
            'Happy',
            'Excited',
            'Angry']

In [16]:
for e in tqdm(emotions):
    data_cls[e] = np.nan

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1002.66it/s]


In [17]:
for i in tqdm(range(len(data_cls))):
    lyrics_emotion = emo_dict[str(i)]['emotion']
    values = [lyrics_emotion[e] for e in emotions]
    data_cls.at[i, emotions] = values
#     break

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4395/4395 [00:06<00:00, 663.21it/s]


## Modeling

### Measures

In [18]:
def measures(ypred, ytest):
    return f1_score(ypred, ytest, average='micro'), cohen_kappa_score(ypred, ytest)

kappa_scorer = make_scorer(cohen_kappa_score)

scoring = kappa_scorer

### SVM

In [19]:
def svm_model(xtrain, xtest, ytrain, ytest, batch):
    t0 = time()
    param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], 
                 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
    clf = GridSearchCV(
        SVC(), param_grid, scoring=scoring
    )
    searcher = clf.fit(xtrain, ytrain)
    estimator = searcher.best_estimator_
    f1, kappa = measures(estimator.predict(xtest), ytest)
    print('[SVM] training: ', 'f1:', f1, 'kappa:', kappa, "done in %0.3fs." % (time() - t0))
    with open(batch + '_svm_clf.pickle', 'wb') as f:
        pickle.dump(clf, f)
    with open(batch + '_svm_searcher.pickle',
              'wb') as sf:
        pickle.dump(searcher, sf)
    with open(batch + '_svm_estimator.pickle',
      'wb') as sfm:
        pickle.dump(estimator, sfm)
    return searcher, estimator, clf, f1, kappa

### Logistic Regression

In [20]:
def lg_model(xtrain, xtest, ytrain, ytest, batch):
    t0 = time()
    param_grid = {'penalty': ['l1', 'l2', 'elasticnet'],
                  'max_iter': range(10, 50, 10)}
    clf = GridSearchCV(
        LogisticRegression(multi_class='auto', n_jobs=-1), param_grid, scoring=scoring
    )
    searcher = clf.fit(xtrain, ytrain)
    estimator = searcher.best_estimator_
    f1, kappa = measures(estimator.predict(xtest), ytest)
    print('[LG] training: ', 'f1:', f1, 'kappa:', kappa, "done in %0.3fs." % (time() - t0))
    with open(batch + '_lg_clf.pickle', 'wb') as f:
        pickle.dump(clf, f)
    with open(batch + '_lg_searcher.pickle',
              'wb') as sf:
        pickle.dump(searcher, sf)
    with open(batch + '_lg_estimator.pickle',
      'wb') as sfm:
        pickle.dump(estimator, sfm)
    return searcher, estimator, clf, f1, kappa

### Decision Tree

In [21]:
def dt_model(xtrain, xtest, ytrain, ytest, batch):
    t0 = time()
    param_grid = {'min_samples_split': range(2, 403, 20)}
    clf = GridSearchCV(
        tree.DecisionTreeClassifier(), param_grid, scoring=scoring
    )
    searcher = clf.fit(xtrain, ytrain)
    estimator = searcher.best_estimator_
    f1, kappa = measures(estimator.predict(xtest), ytest)
    print('[dt] training: ', 'f1:', f1, 'kappa:', kappa, "done in %0.3fs." % (time() - t0))
    with open(batch + '_dt_clf.pickle', 'wb') as f:
        pickle.dump(clf, f)
    with open(batch + '_dt_searcher.pickle',
              'wb') as sf:
        pickle.dump(searcher, sf)
    with open(batch + '_dt_estimator.pickle',
      'wb') as sfm:
        pickle.dump(estimator, sfm)
    return searcher, estimator, clf, f1, kappa

### Naive Bayes

In [22]:
def nb_model(xtrain, xtest, ytrain, ytest, batch):
    t0 = time()
    param_grid = {}
    clf = GridSearchCV(GaussianNB(), param_grid)
    searcher = clf.fit(xtrain, ytrain)
    estimator = searcher.best_estimator_
    f1, kappa = measures(estimator.predict(xtest), ytest)
    print('[NB] training: ', 'f1:', f1, 'kappa:', kappa, "done in %0.3fs." % (time() - t0))
    with open(batch + '_nb_clf.pickle', 'wb') as f:
        pickle.dump(clf, f)
    with open(batch + '_nb_searcher.pickle',
              'wb') as sf:
        pickle.dump(searcher, sf)
    with open(batch + '_nb_estimator.pickle',
      'wb') as sfm:
        pickle.dump(estimator, sfm)
    return searcher, estimator, clf, f1, kappa

### NN

In [23]:
def nn_model(xtrain, xtest, ytrain, ytest, batch):
    t0 = time()
    hidden_layer_sizes = (10, 50, 50, 10)
    param_grid = {
                  'alpha': [1e-5, 1e-4, 1e-3],
                  'solver': ['lbfgs', 'sgd', 'adam'],
                  'activation': ['identity', 'logistic', 'tanh', 'relu'],
                 }
    if batch == 'lyrics':
        hidden_layer_sizes=(100, 200, 100, 50)
    clf = GridSearchCV(MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=905, early_stopping=True), param_grid)
    searcher = clf.fit(xtrain, ytrain)
    estimator = searcher.best_estimator_
    f1, kappa = measures(estimator.predict(xtest), ytest)
    print('[nn] training: ', 'f1:', f1, 'kappa:', kappa, "done in %0.3fs." % (time() - t0))
    with open(batch + '_nn_clf.pickle', 'wb') as f:
        pickle.dump(clf, f)
    with open(batch + '_nn_searcher.pickle',
              'wb') as sf:
        pickle.dump(searcher, sf)
    with open(batch + '_nn_estimator.pickle',
      'wb') as sfm:
        pickle.dump(estimator, sfm)
    return searcher, estimator, clf, f1, kappa

## Data spliting

In [24]:
data_cls['genre'].value_counts()

Rock             1840
Pop               657
Hip-Hop           472
Metal             368
Not Available     325
Country           283
Electronic        135
Jazz              110
R&B                67
Other              60
Indie              54
Folk               24
Name: genre, dtype: int64

In [25]:
target = {}
count = 1
for i in set(data_cls['genre'].tolist()):
    target[i] = count
    count += 1

In [26]:
target

{'Folk': 1,
 'Pop': 2,
 'Country': 3,
 'Other': 4,
 'Electronic': 5,
 'Jazz': 6,
 'Hip-Hop': 7,
 'Indie': 8,
 'Not Available': 9,
 'Metal': 10,
 'R&B': 11,
 'Rock': 12}

In [27]:
data_cls['genre_'] = data_cls['genre']

data_cls['genre'] = data_cls['genre'].map(target)

In [28]:
data_cls.columns

Index(['index', 'song', 'year', 'artist', 'genre', 'lyrics', 'word_count',
       'represent', 'represent_2nd', 'word_count_2nd', 'clustered_label',
       'Fear', 'Sad', 'Bored', 'Happy', 'Excited', 'Angry', 'genre_'],
      dtype='object')

# Training

## Emotion & Topic

In [29]:
X = data_cls[['clustered_label','Fear', 'Sad', 'Bored', 'Happy', 'Excited', 'Angry']].values
Y = data_cls['genre'].astype(int).values

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=31)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((3516, 7), (879, 7), (3516,), (879,))

In [30]:
# nn_searcher, nn_estimator, nn_clf, nn_f1, nn_kappa = nn_model(xtrain, xtest, ytrain, ytest, batch='Combined')
# svm_searcher, svm_estimator, svm_clf, svm_f1, svm_kappa = svm_model(xtrain, xtest, ytrain, ytest, batch='Combined')
lg_searcher, lg_estimator, lg_clf, lg_f1, lg_kappa = lg_model(xtrain, xtest, ytrain, ytest, batch='Combined')
dt_searcher, dt_estimator, dt_clf, dt_f1, dt_kappa = dt_model(xtrain, xtest, ytrain, ytest, batch='Combined')
nb_searcher, nb_estimator, nb_clf, nb_f1, nb_kappa = nb_model(xtrain, xtest, ytrain, ytest, batch='Combined')

D:\Softwares\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\Softwares\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got elasticnet penalty.

  FitFailedWarning)
D:\Softwares\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\Softwares\Anaconda3\lib\site-packages\

[LG] training:  f1: 0.4641638225255973 kappa: 0.15662068923370254 done in 3.476s.
[dt] training:  f1: 0.4061433447098976 kappa: 0.13125353822035324 done in 2.095s.
[NB] training:  f1: 0.43913538111490324 kappa: 0.1412426355621348 done in 0.019s.


### Lyrics

In [31]:
X_ = data
Y_ = data_cls['genre'].astype(int).values

xtrain_, xtest_, ytrain_, ytest_ = train_test_split(X_, Y_, test_size=0.2, random_state=31)
xtrain_.shape, xtest_.shape, ytrain_.shape, ytest_.shape

((3516, 788), (879, 788), (3516,), (879,))

In [32]:
# nn_searcher_, nn_estimator_, nn_clf_, nn_f1_, nn_kappa_ = nn_model(xtrain_, xtest_, ytrain_, ytest_, batch='lyrics')
# svm_searcher_, svm_estimator_, svm_clf_, svm_f1_, svm_kappa_ = svm_model(xtrain_, xtest_, ytrain_, ytest_, batch='lyrics')
lg_searcher_, lg_estimator_, lg_clf_, lg_f1_, lg_kappa_ = lg_model(xtrain_, xtest_, ytrain_, ytest_, batch='lyrics')
dt_searcher_, dt_estimator_, dt_clf_, dt_f1_, dt_kappa_ = dt_model(xtrain_, xtest_, ytrain_, ytest_, batch='lyrics')
nb_searcher_, nb_estimator_, nb_clf_, nb_f1_, nb_kappa_ = nb_model(xtrain_, xtest_, ytrain_, ytest_, batch='lyrics')

D:\Softwares\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\Softwares\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got elasticnet penalty.

  FitFailedWarning)
D:\Softwares\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\Softwares\Anaconda3\lib\site-packages\

[LG] training:  f1: 0.5164960182025028 kappa: 0.2754150252826477 done in 18.252s.
[dt] training:  f1: 0.41069397042093286 kappa: 0.18626820219176898 done in 67.965s.
[NB] training:  f1: 0.24118316268486917 kappa: 0.13869231865385823 done in 0.858s.


# Test

In [35]:
def load_models(path):
    models = {}
    for root, dirs, files in os.walk(path):
        for fn in files:
            eachpath = str(root+'/'+fn)
            if '.pickle' in eachpath:
                print(eachpath)
                models[fn.replace('.pickle', '')] = pickle.load(open(eachpath, 'rb'))
    return models

In [38]:
models_path = r'./'
models = load_models(models_path)
models.keys()

.//.gitignore


UnpicklingError: invalid load key, '*'.

In [34]:
evaluate_results = pd.DataFrame(columns=['group_model_k', 'f1', 'kappa'])
models_10fold = []
for model in list(models.keys()):
    for i in range(1, 11):
        model_i = model + '_' + str(i)
        models_10fold.append(model_i)
evaluate_results['group_model_k'] = pd.Series(list(models_10fold))
evaluate_results.head()

NameError: name 'models' is not defined